In [ ]:
from utils import *

import pickle
import datetime

from scipy.optimize import minimize, root

import matplotlib.image as mpimg
from matplotlib.gridspec import GridSpec


# Figure 1

In [ ]:
def main(Vg, Vb, params):
    with ProcessPoolExecutor() as executor:
        results = np.array(list(tqdm(executor.map(solve_bias_gate,Vg,Vb,params),total=len(Vg))),dtype=object)
    return results

def power(Vs,params):
    vg,vb = Vs
    system = anderson(vg,vb,params)
    system.solve()
    return vb*np.sum(system.current[[0,2]])

def load_voltage(vb,R,system):
    system.change(mulst={0:vb/2, 1:-vb/2, 2:vb/2, 3:-vb/2})
    system.solve()
    return np.sum(system.current[[0,2]])-vb/R

# Detailed power plot

In [ ]:
n_vg_stab,n_vb_stab = 200,200
Vg_stab=np.linspace(-15,15,n_vg_stab)
Vb_stab=np.linspace(-1.05,1.05,n_vb_stab)
dT=0.3
params_rtd=Parameters(B=0., u=100., Tl=1.+dT, Tr=1., gamma=.25, kerntype='RTD', dband=1e4, countingleads=[0,2])

In [ ]:
if __name__ == '__main__':
    results = pstab_std(Vg_stab,Vb_stab,params_rtd)

In [ ]:
# pickle the results
timestamp = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
with open(timestamp+'fig1heatmap.pkl', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
# unpickle the results
with open(timestamp+'fig1heatmap.pkl', 'rb') as f:
    results = pickle.load(f)

In [ ]:
I_stab=np.array([ri[0][0]+ri[0][2] for ri in results]).reshape(n_vg_stab,n_vb_stab)
Q_stab=np.array([ri[1][0]+ri[1][2] for ri in results]).reshape(n_vg_stab,n_vb_stab)
Vb_rep=np.repeat(Vb_stab,n_vg_stab).reshape(n_vb_stab,n_vg_stab)
P_stab=-I_stab*Vb_rep
P_stab[P_stab<0]=np.nan

In [ ]:
# max power point
system = anderson(0,0,params_rtd)
v_max = minimize(power, [-1,-1], args=(params_rtd,))
# corresponding load voltage and current give load resistance
# RTD
system = anderson(v_max.x[0],v_max.x[1],params_rtd)
system.solve()
I_max = system.current[0]+system.current[2]
R_load = v_max.x[1]/I_max


In [ ]:
# thermowiggle
Vg=np.linspace(Vg_stab.min(),Vg_stab.max(),100)
Vb=[]
for i,vg in tqdm(enumerate(Vg),desc='Finding Vb',total=len(Vg)):
    system = anderson(vg,0,params_rtd)
    Vb.append(root(lambda vb: load_voltage(vb,R_load,system),0))

Vb_load=np.array([vb.x for vb in Vb]).reshape(len(Vg),)

# Plot

In [ ]:
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "dejavuserif"
fig,ax = plt.subplots(figsize=(4,4),constrained_layout=True)
P_plot = ax.imshow(P_stab/(params_rtd.gamma*params_rtd.Tr)*1e3,extent=[Vg_stab[0],Vg_stab[-1],Vb_stab[0],Vb_stab[-1]],aspect='auto',origin='lower',cmap='inferno',vmin=0)
# colorbar
cbar = plt.colorbar(P_plot,location='top')
cbar.set_label(r'Power $P/(\Gamma \cdot T) \cdot 10^3$',fontsize=12)
# thermowiggle
plt.plot(Vg,Vb_load,'r--',lw=2.5)
plt.xlabel(r'Gate voltage $V_g  / T$',fontsize=12)
plt.ylabel(r'Bias voltage $V_b  / T$',fontsize=12)
plt.plot(v_max.x[0],v_max.x[1],markerfacecolor='r',marker='X', markersize=12, markeredgecolor='k', markeredgewidth=1, label='Max power point')
fig.savefig('fig1c.svg',dpi=300)
plt.show()
